The MNIST classifier:
0. Importing packages (Video 1)
1. Define the Model Class (Video 1)
2. Define a function for checking accuracy (Video 4)
3. Download MNIST data and define the dataloader (Video 2)
4. Define hyperparameters, device, model, optimizer, loss function (Video 2)
5. Define the training loop (Video 3)


* Importing packages

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
from torch.utils.data import DataLoader

* Defining the Model Class:

In [2]:
class MyImageClassifier(nn.Module):
  def __init__(self, num_pixels, num_out_1, num_classes):
    super(MyImageClassifier, self).__init__()
    self.my_image_classifier = nn.Sequential(
        nn.Linear(num_pixels, num_out_1),
        nn.ReLU(),
        nn.Linear(num_out_1, num_classes)
    )
  
  def forward(self, x):
    x = self.my_image_classifier(x)
    return x

In [3]:
model = MyImageClassifier(500, 50, 10)
x = torch.randn(35, 1, 25, 20)
x = x.reshape(-1, 500)
y = model(x)
print(y.shape)

torch.Size([35, 10])


* Define a function for checking accuracy

In [20]:
def acc_calc(loader, model):
  num_corrects = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      # send the data to the device
      x = x.to(device)
      y = y.to(device)

      # prepare the data for the model
      x = x.reshape(-1, 784)

      # forward
      y_hat = model(x)

      # calculations for accuracy
      _, predictions = y_hat.max(1)
      num_corrects += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Accuracy = {num_corrects/num_samples*100:.2f}; Received {num_corrects}/{num_samples}")
    model.train()


* Define hyperparameters, device, model, optimizer, loss function


In [19]:
# hyperparameters
LR = 3e-4
BATCH_SIZE = 128
NUM_EPOCHS = 5
NUM_OUT_1 = 50

# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# model
model = MyImageClassifier(num_pixels=784, num_out_1=NUM_OUT_1, num_classes=10)
model = model.to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=LR)

# loss function
criterion = nn.CrossEntropyLoss()


cuda


* Download MNIST data and define the dataloader

In [17]:
import torchvision.transforms as transforms

# download data
mnist_train = datasets.MNIST(root="/datasets", transform=transforms.ToTensor(), download=True, train=True)
mnist_test = datasets.MNIST(root="/datasets", transform=transforms.ToTensor(), download=True, train=False)

# defining the dataloder
train_loader = DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=BATCH_SIZE, shuffle=True)


* Define the training loop (testing embedded)

In [21]:
for epoch in range(NUM_EPOCHS):
  running_loss = 0.0
  for batch_idx, (x, y) in enumerate(train_loader):
    # send the data to the device
    x = x.to(device)
    y = y.to(device)

    # prepare the data for the model
    x = x.reshape(-1, 784)

    # forward
    y_hat = model(x)
    loss = criterion(y_hat, y)
    running_loss += loss

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch + 1} is done; loss = {running_loss}")
  acc_calc(test_loader, model)

Epoch 1 is done; loss = 408.29583740234375
Accuracy = 89.86; Received 8986/10000
Epoch 2 is done; loss = 166.52503967285156
Accuracy = 91.57; Received 9157/10000
Epoch 3 is done; loss = 137.46730041503906
Accuracy = 92.49; Received 9249/10000
Epoch 4 is done; loss = 121.78385925292969
Accuracy = 93.27; Received 9327/10000
Epoch 5 is done; loss = 110.26744842529297
Accuracy = 93.72; Received 9372/10000
